# 🧬 Facial AI Platform — DECA Face Reconstruction

This notebook reconstructs a 3D FLAME mesh + texture from your photos.

**What you need:**
1. 1-3 face photos (front required, left 45° and right 45° optional)
2. FLAME 2020 model file (download from https://flame.is.tue.mpg.de/)

**What you get:**
- `face_mesh.obj` — FLAME topology 3D mesh
- `face_texture.png` — 1024x1024 albedo texture map
- `face_normal.png` — Normal map for surface detail
- `face_params.json` — FLAME shape/expression parameters

Upload these files to your Facial AI Platform web app.

## Step 1: Setup Environment

In [ ]:
# Check GPU
!nvidia-smi
import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None"}')

In [ ]:
# Install DECA dependencies
!pip install -q torch torchvision
!pip install -q face-alignment opencv-python-headless scikit-image
!pip install -q pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu121_pyt241/download.html
!pip install -q chumpy

# Clone DECA
!git clone https://github.com/yfeng95/DECA.git
%cd DECA
!pip install -q -r requirements.txt

print('✅ Dependencies installed')

In [ ]:
# Download DECA pretrained model
!mkdir -p data
!gdown --id 1rp8kdyLPvErw2dTmqtjISRVvQLj6Yzje -O data/deca_model.tar

# You need to upload the FLAME model manually
# Download from: https://flame.is.tue.mpg.de/
# Upload generic_model.pkl to DECA/data/

print('✅ DECA model downloaded')
print('⚠️  Now upload FLAME model (generic_model.pkl) to DECA/data/')

## Step 2: Upload FLAME Model

Upload the `generic_model.pkl` file you downloaded from flame.is.tue.mpg.de

In [ ]:
from google.colab import files
import os

# Check if FLAME model already exists
flame_path = 'data/generic_model.pkl'
if not os.path.exists(flame_path):
    print('Please upload generic_model.pkl from FLAME website:')
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, flame_path)
        print(f'✅ FLAME model saved to {flame_path}')
else:
    print(f'✅ FLAME model already exists at {flame_path}')

## Step 3: Upload Your Face Photos

Upload 1-3 photos:
- **Required:** Front-facing, neutral expression
- **Optional:** Left 45°, Right 45°

Tips:
- Diffuse, even lighting (no harsh shadows)
- Neutral expression, mouth closed
- Hair pulled back from face
- High resolution

In [ ]:
from google.colab import files
import shutil

# Create input directory
INPUT_DIR = '/content/face_input'
os.makedirs(INPUT_DIR, exist_ok=True)

print('📸 Upload your face photos (1-3 images):')
uploaded = files.upload()

for filename, data in uploaded.items():
    dest = os.path.join(INPUT_DIR, filename)
    with open(dest, 'wb') as f:
        f.write(data)
    print(f'  ✅ Saved: {filename} ({len(data)/1024:.0f} KB)')

print(f'\n📁 {len(uploaded)} photo(s) uploaded to {INPUT_DIR}')

## Step 4: Run DECA Reconstruction

In [ ]:
import sys
sys.path.insert(0, '/content/DECA')

import cv2
import numpy as np
from PIL import Image
import json

# Run DECA reconstruction
OUTPUT_DIR = '/content/face_output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

from decalib.deca import DECA
from decalib.utils.config import cfg as deca_cfg
from decalib.datasets import datasets

# Initialize DECA
deca_cfg.model.use_tex = True
deca_cfg.rasterizer_type = 'pytorch3d'
deca = DECA(config=deca_cfg, device='cuda')

print('✅ DECA model loaded')

# Process each photo
input_files = sorted([f for f in os.listdir(INPUT_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
print(f'Processing {len(input_files)} image(s)...')

all_params = []

for i, filename in enumerate(input_files):
    img_path = os.path.join(INPUT_DIR, filename)
    print(f'\n--- Processing {filename} ({i+1}/{len(input_files)}) ---')

    # Load and preprocess
    testdata = datasets.TestData(img_path, iscrop=True, face_detector='fan', sample_step=1)
    if len(testdata) == 0:
        print(f'  ⚠️ No face detected in {filename}, skipping')
        continue

    images = testdata[0]['image'].unsqueeze(0).to('cuda')

    with torch.no_grad():
        codedict = deca.encode(images)
        opdict, visdict = deca.decode(codedict)

    # Extract parameters
    params = {
        'shape': codedict['shape'].cpu().numpy().tolist()[0],
        'exp': codedict['exp'].cpu().numpy().tolist()[0],
        'pose': codedict['pose'].cpu().numpy().tolist()[0],
        'cam': codedict['cam'].cpu().numpy().tolist()[0],
        'light': codedict['light'].cpu().numpy().tolist()[0] if 'light' in codedict else None,
        'tex': codedict['tex'].cpu().numpy().tolist()[0] if 'tex' in codedict else None,
        'source_image': filename
    }
    all_params.append(params)

    # Get mesh vertices and faces
    vertices = opdict['verts'].cpu().numpy()[0]
    faces = deca.flame.faces_tensor.cpu().numpy()

    print(f'  Mesh: {vertices.shape[0]} vertices, {faces.shape[0]} faces')
    print(f'  Shape params: {len(params["shape"])}')
    print(f'  Expression params: {len(params["exp"])}')

print(f'\n✅ Reconstruction complete for {len(all_params)} image(s)')

## Step 5: Export Results

In [ ]:
# Use the first (front) image as primary reconstruction
primary_idx = 0

# Re-run decode for primary image to get mesh data
testdata = datasets.TestData(
    os.path.join(INPUT_DIR, input_files[primary_idx]),
    iscrop=True, face_detector='fan', sample_step=1
)
images = testdata[0]['image'].unsqueeze(0).to('cuda')

with torch.no_grad():
    codedict = deca.encode(images)
    opdict, visdict = deca.decode(codedict)

vertices = opdict['verts'].cpu().numpy()[0]
faces = deca.flame.faces_tensor.cpu().numpy()

# Get UV coordinates from FLAME
try:
    uvs = deca.flame.vt.cpu().numpy() if hasattr(deca.flame, 'vt') else None
    uv_faces = deca.flame.ft.cpu().numpy() if hasattr(deca.flame, 'ft') else None
except:
    uvs = None
    uv_faces = None

# === Export OBJ mesh ===
obj_path = os.path.join(OUTPUT_DIR, 'face_mesh.obj')
mtl_path = os.path.join(OUTPUT_DIR, 'face_mesh.mtl')

with open(obj_path, 'w') as f:
    f.write('# DECA FLAME Reconstruction\n')
    f.write(f'# Vertices: {vertices.shape[0]}\n')
    f.write(f'# Faces: {faces.shape[0]}\n')
    f.write(f'mtllib face_mesh.mtl\n')
    f.write(f'usemtl face_material\n\n')

    # Vertices
    for v in vertices:
        f.write(f'v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n')

    # UVs
    if uvs is not None:
        for uv in uvs:
            f.write(f'vt {uv[0]:.6f} {uv[1]:.6f}\n')

    # Faces (1-indexed)
    if uvs is not None and uv_faces is not None:
        for fi, face in enumerate(faces):
            uv_face = uv_faces[fi] if fi < len(uv_faces) else face
            f.write(f'f {face[0]+1}/{uv_face[0]+1} {face[1]+1}/{uv_face[1]+1} {face[2]+1}/{uv_face[2]+1}\n')
    else:
        for face in faces:
            f.write(f'f {face[0]+1} {face[1]+1} {face[2]+1}\n')

# MTL file
with open(mtl_path, 'w') as f:
    f.write('newmtl face_material\n')
    f.write('Ka 0.2 0.2 0.2\n')
    f.write('Kd 0.8 0.8 0.8\n')
    f.write('map_Kd face_texture.png\n')
    f.write('bump face_normal.png\n')

print(f'✅ Mesh exported: {obj_path}')
print(f'   {vertices.shape[0]} vertices, {faces.shape[0]} faces')

# === Export texture ===
tex_path = os.path.join(OUTPUT_DIR, 'face_texture.png')

if 'uv_texture_gt' in visdict:
    texture = visdict['uv_texture_gt'][0].cpu().numpy()
    texture = (texture.transpose(1, 2, 0) * 255).astype(np.uint8)
    Image.fromarray(texture).save(tex_path)
    print(f'✅ Texture exported: {tex_path} ({texture.shape[0]}x{texture.shape[1]})')
elif opdict.get('albedo') is not None:
    albedo = opdict['albedo'][0].cpu().numpy()
    albedo = (albedo.transpose(1, 2, 0) * 255).astype(np.uint8)
    Image.fromarray(albedo).save(tex_path)
    print(f'✅ Albedo texture exported: {tex_path}')
else:
    # Fallback: use input photo directly
    src_img = cv2.imread(os.path.join(INPUT_DIR, input_files[primary_idx]))
    src_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)
    src_img = cv2.resize(src_img, (1024, 1024))
    Image.fromarray(src_img).save(tex_path)
    print(f'✅ Fallback texture from photo: {tex_path}')

# === Export normal map from displacement ===
normal_path = os.path.join(OUTPUT_DIR, 'face_normal.png')

if 'displacement_map' in opdict:
    disp = opdict['displacement_map'][0].cpu().numpy()
    disp = (disp.transpose(1, 2, 0) * 127.5 + 127.5).astype(np.uint8)
    Image.fromarray(disp).save(normal_path)
    print(f'✅ Normal/displacement map exported: {normal_path}')
else:
    # Generate normal map from vertices
    normal_img = np.full((1024, 1024, 3), 128, dtype=np.uint8)
    normal_img[:, :, 2] = 255  # Z-up default normal
    Image.fromarray(normal_img).save(normal_path)
    print(f'✅ Default normal map exported: {normal_path}')

# === Export parameters JSON ===
params_path = os.path.join(OUTPUT_DIR, 'face_params.json')
export_data = {
    'vertex_count': int(vertices.shape[0]),
    'face_count': int(faces.shape[0]),
    'shape_params': all_params[primary_idx]['shape'],
    'expression_params': all_params[primary_idx]['exp'],
    'pose_params': all_params[primary_idx]['pose'],
    'source_images': [p['source_image'] for p in all_params],
    'reconstruction_method': 'DECA',
    'flame_model': 'FLAME2020'
}

with open(params_path, 'w') as f:
    json.dump(export_data, f, indent=2)

print(f'✅ Parameters exported: {params_path}')
print(f'\n📁 All files in {OUTPUT_DIR}:')
for f in os.listdir(OUTPUT_DIR):
    size = os.path.getsize(os.path.join(OUTPUT_DIR, f))
    print(f'  {f} ({size/1024:.0f} KB)')

## Step 6: Preview Reconstruction

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(16, 5))

# Show source photo
ax1 = fig.add_subplot(141)
src = Image.open(os.path.join(INPUT_DIR, input_files[primary_idx]))
ax1.imshow(src)
ax1.set_title('Source Photo')
ax1.axis('off')

# Show 3D mesh
ax2 = fig.add_subplot(142, projection='3d')
ax2.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2],
                 triangles=faces, color='#e8b89d', edgecolor='gray',
                 linewidth=0.1, alpha=0.8)
ax2.set_title('3D Mesh (Front)')
ax2.view_init(elev=0, azim=0)
ax2.axis('off')

# Show texture
ax3 = fig.add_subplot(143)
tex = Image.open(tex_path)
ax3.imshow(tex)
ax3.set_title('Texture Map')
ax3.axis('off')

# Show 3D mesh (profile)
ax4 = fig.add_subplot(144, projection='3d')
ax4.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2],
                 triangles=faces, color='#e8b89d', edgecolor='gray',
                 linewidth=0.1, alpha=0.8)
ax4.set_title('3D Mesh (Profile)')
ax4.view_init(elev=0, azim=90)
ax4.axis('off')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'preview.png'), dpi=150)
plt.show()
print('✅ Preview saved')

## Step 7: Download Results

Download the ZIP file and upload the contents to your Facial AI Platform.

In [ ]:
import zipfile

zip_path = '/content/facial_reconstruction.zip'

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    for f in os.listdir(OUTPUT_DIR):
        zf.write(os.path.join(OUTPUT_DIR, f), f)

zip_size = os.path.getsize(zip_path) / (1024 * 1024)
print(f'📦 Created: facial_reconstruction.zip ({zip_size:.1f} MB)')
print(f'\nContents:')
with zipfile.ZipFile(zip_path, 'r') as zf:
    for info in zf.infolist():
        print(f'  {info.filename} ({info.file_size/1024:.0f} KB)')

print(f'\n⬇️  Downloading...')
files.download(zip_path)

print('\n✅ Done! Upload these files to your Facial AI Platform at:')
print('   https://facial-ai-project.vercel.app')